# 10. WORKING WITH INDEXES

(_1h35min_)

## 10.1 Module introduction

- What are indexes?
- Different types of indexes
- Using & optimizing indexes

---

## 10.2 What are indexes and why we use them?

- [Create Index](https://www.mongodb.com/docs/atlas/atlas-search/create-index/)

<img src="..\imgs\s10\s10-1.png" width=700 height=400 >

- Cada vez que um novo documento é inserido na collection, o index é atualizado
- Então se há muitos indexes em uma coleção, os inserts se tornarão mais demorados/terão menos performance

<img src="..\imgs\s10\s10-2.png" width=700 height=400 >

---

## 10.3 Adding a single field index

- [Explain](https://www.mongodb.com/docs/manual/reference/command/explain/#explain)
- [Create Index](https://www.mongodb.com/docs/manual/reference/command/createIndexes/)

> The [**`explain`**](https://www.mongodb.com/docs/manual/reference/command/explain/#mongodb-dbcommand-dbcmd.explain) command provides information on the execution of the following commands:
 [**`aggregate`**](https://www.mongodb.com/docs/manual/reference/command/aggregate/#mongodb-dbcommand-dbcmd.aggregate), [**`count`**](https://www.mongodb.com/docs/manual/reference/command/count/#mongodb-dbcommand-dbcmd.count), [**`distinct`**](https://www.mongodb.com/docs/manual/reference/command/distinct/#mongodb-dbcommand-dbcmd.distinct), [**`find`**](https://www.mongodb.com/docs/manual/reference/command/find/#mongodb-dbcommand-dbcmd.find), [**`findAndModify`**](https://www.mongodb.com/docs/manual/reference/command/findAndModify/#mongodb-dbcommand-dbcmd.findAndModify), [**`delete`**](https://www.mongodb.com/docs/manual/reference/command/delete/#mongodb-dbcommand-dbcmd.delete), [**`mapReduce`**](https://www.mongodb.com/docs/manual/reference/command/mapReduce/#mongodb-dbcommand-dbcmd.mapReduce), and [**`update`.**](https://www.mongodb.com/docs/manual/reference/command/update/#mongodb-dbcommand-dbcmd.update)

- https://www.mongodb.com/docs/manual/reference/explain-results/
- **`executionStats` Mode** → https://www.mongodb.com/docs/manual/reference/command/explain/#executionstats-mode

In [ ]:
mongoimport persons.json -d contactData -c contacts --jsonArray --drop

In [ ]:
db.contacts.find( { "dob.age": { $gt: 60 } } )

db.contacts.explain().find( { "dob.age": { $gt: 60 } } )

db.contacts.explain("executionStats").find( { "dob.age": { $gt: 60 } } ) // also show the time the query took to return results

In [ ]:
db.contacts.createIndex( { "dob.age": 1 } )

db.contacts.explain("executionStats").find( { "dob.age": { $gt: 60 } } )

---

## 10.4 Indexes behind the scene

> What does `createIndex()` do in detail?
> 
> 
> Whilst we can't really see the index, you can think of the index as a simple list of values + pointers to the original document.
> 
> Something like this (for the "age" field):
> 
> `(29, "address in memory/ collection a1")`
> 
> `(30, "address in memory/ collection a2")`
> 
> `(33, "address in memory/ collection a3")`
> 
> The documents in the collection would be at the "addresses" `a1`, `a2` and `a3`. The order does not have to match the order in the index (and most likely, it indeed won't).
> 
> The important thing is that the index items are **ordered** (ascending or descending - depending on how you created the index). 
> `createIndex({age: 1})` creates an index with **ascending sorting**, `createIndex({age: -1})` creates one with **descending** **sorting**.
> 
> MongoDB is now able to quickly find a fitting document when you filter for its age as it has a sorted list. Sorted lists are way quicker to search because you can skip entire ranges (and don't have to look at every single document).
> 
> Additionally, sorting (via `sort(...)`) will also be sped up because you already have a sorted list. Of course this is only true when sorting for the age.
>

---

## 10.5 Understanding indexes restrictions

- https://www.mongodb.com/docs/manual/reference/command/dropIndexes/ 

In [ ]:
db.contacts.dropIndex( { "dob.age": 1 } )

---

## 10.6 Creating compounding indexes 

- [Compound Index](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-compound/#compound-indexes)

> Compound indexes collect and sort data from two or more fields in each document in a collection. 
Data is grouped by the first field in the index and then by each subsequent field.
> 
> 
> For example, the following image shows a compound index where documents are first grouped by `userid` in ascending order (alphabetically).
> Then, the `scores` for each `userid` are sorted in descending order:
> 
> <img src="..\imgs\s10\s10-3.png" width=600 height=200 >
>

In [ ]:
db.contacts.createIndex({gender: 1})
db.contacts.explain("executionStats").find({ gender: "male" })
db.contacts.dropIndex({gender: 1})

// compound indexes:
// cria 1 index com a combinação de 2 elementos, neste caso: "age" e "gender":
db.contacts.createIndex( { "dob.age": 1, gender: 1 } ) // dob.age_1_gender_1

// usando o index em numa query:
db.contacts.explain().find( { "dob.age": 35, gender: "male" } )

// o index pode ser usado para consultas que estão procurando ambos os campos do index, ou para consultas que procuram pelo primeiro campo
// lendo "da esquerda para a direita", neste caso, consultas somente pelo "dob.age"
// consultas somente pelo "gender", neste caso, não utilizaram do index
db.contacts.explain().find( { "dob.age": 35 } )

---

## 10.7 Using indexes for sorting

- [Use Indexes to Sort Query Results](https://www.mongodb.com/docs/manual/tutorial/sort-results-with-indexes/#use-indexes-to-sort-query-results)

In [ ]:
db.contacts.explain().find( { "dob.age": 35 } ).sort( { gender: 1 } ) 

---

## 10.8 Understanding the default index 

- [getIndexes](https://www.mongodb.com/docs/manual/reference/method/db.collection.getIndexes)

> Returns an array that holds a list of documents that identify and describe the existing indexes on the collection, including [**hidden indexes**](https://www.mongodb.com/docs/manual/core/index-hidden/#std-label-index-type-hidden) and indexes that are currently being built.
> 
> 
> You must call [**`db.collection.getIndexes()`**](https://www.mongodb.com/docs/manual/reference/method/db.collection.getIndexes/#mongodb-method-db.collection.getIndexes) on a collection. For example:
>

In [ ]:
db.contacts.getIndexes()

---

## 10.9 Configuring indexes 

- [Command fields](https://www.mongodb.com/docs/manual/reference/command/createIndexes/#command-fields)

> 
> 
> 
> 
> | `unique` | boolean | Optional. Creates a unique index so that the collection will not accept insertion or update of documents where the index key value matches an existing value in the index.
> Specify `true` to create a unique index. The default value is `false`.
> The option is *unavailable* for [**hashed**](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-hashed/#std-label-index-hashed-index) indexes. |
> | --- | --- | --- |

In [ ]:
db.contacts.findOne()

db.contacts.createIndex( { email: 1 }, { unique: true } ) // no exemplo, acha um erro pq o email "abigail.clark@example.com" já é duplicado na collection

---

## 10.10 Understanding partial indexes

- [Partial Index](https://www.mongodb.com/docs/manual/core/index-partial/)

> Partial indexes only index the documents in a collection that meet a specified filter expression. 
By indexing a subset of the documents in a collection, partial indexes have lower storage requirements and reduced performance costs for index creation and maintenance.
>

In [ ]:
db.contacts.getIndexes()
db.contacts.dropIndex( { "dob.age": 1, gender: 1 } )

// criação de partial index:
db.contacts.createIndex( { "dob.age": 1 }, { partialFilterExpression:  { "dob.age": { $gt: 60 } } } ) // dob.age_1

db.contacts.createIndex( { "dob.age": 1 }, { partialFilterExpression:  { gender: "male" } } ) // outra possibilidade

db.contacts.explain().find( { "dob.age": { $gt: 60 } } ) //  stage: 'IXSCAN'

---

## 10.11 Applying the partial index

In [ ]:
db.users.insertMany( [ { name: "Max", email: "max@test.com" }, { name: "Manu" } ] )

db.users.createIndex( { email: 1 } )
db.users.dropIndex( { email: 1 } )

db.users.createIndex( { email: 1 }, { unique: true } )

db.users.insertOne( { name: "Anna" } ) // MongoServerError: E11000 duplicate key error collection: contactData.users index: email_1 dup key: { email: null }
// o erro ocorre pq com essa inserção de "Anna" sem um email, existe o "email: null" 2x
// o mongoDB trata "non existing" values como um valor no index

db.users.dropIndex( { email: 1 } )

// caso o correto seja permitir que o campo seja nulo em alguns documentos, mesmo com a chave de "unique":
db.users.createIndex( { email: 1 }, { unique: true, partialFilterExpression: { email: { $exists: true } } } ) // só cria a chave de "unique" em casos onde o email não seja NULL

db.users.insertOne( { name: "Anna" } ) // works
db.users.insertOne( { name: "Anna" , email: "max@test.com"} ) // MongoServerError: E11000 duplicate key error collection: contactData.users index: email_1 dup key: { email: "max@test.com" }

---

## 10.12 Understanding the Time-To-Live (TTL) index 

- [Index TTL](https://www.mongodb.com/docs/manual/core/index-ttl/)

> TTL indexes are special single-field indexes that MongoDB can use to automatically remove documents from a collection after a certain amount of time or at a specific clock time. 
Data expiration is useful for certain types of information like machine generated event data, logs, and session information that only need to persist in a database for a finite amount of time.
>

In [ ]:
db.sessions.insertOne( { data: "randomText", createdAt: new Date() } )

db.sessions.createIndex( { createdAt: 1 }, { expireAfterSeconds: 10 } )

db.sessions.insertOne( { data: "randomText2", createdAt: new Date() } )

- Inserir um novo documento na collection ativa um trigger do mongoDB para reavaliar a collection e ver se há novos documentos expirados para exclusão, inclusive documentos criados antes da criação da regra do TTL

---

## 10.13 Query diagnosis and query planning

- [Explain](https://www.mongodb.com/docs/manual/reference/command/explain/)
- [Explain Results](https://www.mongodb.com/docs/manual/reference/explain-results/)

| **Field** | **Type** | **Description** |
| --- | --- | --- |
| `explain` | document | A document specifying the command for which to return the execution information.<br>For details on the specific command document, see [**`aggregate`**](https://www.mongodb.com/docs/manual/reference/command/aggregate/#mongodb-dbcommand-dbcmd.aggregate), [**`count`**](https://www.mongodb.com/docs/manual/reference/command/count/#mongodb-dbcommand-dbcmd.count), [**`distinct`**](https://www.mongodb.com/docs/manual/reference/command/distinct/#mongodb-dbcommand-dbcmd.distinct), [**`find`**](https://www.mongodb.com/docs/manual/reference/command/find/#mongodb-dbcommand-dbcmd.find), [**`findAndModify`**](https://www.mongodb.com/docs/manual/reference/command/findAndModify/#mongodb-dbcommand-dbcmd.findAndModify), [**`delete`**](https://www.mongodb.com/docs/manual/reference/command/delete/#mongodb-dbcommand-dbcmd.delete), [**`mapReduce`**](https://www.mongodb.com/docs/manual/reference/command/mapReduce/#mongodb-dbcommand-dbcmd.mapReduce), and [**`update`.**](https://www.mongodb.com/docs/manual/reference/command/update/#mongodb-dbcommand-dbcmd.update) |
| `verbosity` | string | Optional.<br>A string specifying the mode in which to run [**`explain`**](https://www.mongodb.com/docs/manual/reference/command/explain/#mongodb-dbcommand-dbcmd.explain).<br>The mode affects the behavior of [**`explain`**](https://www.mongodb.com/docs/manual/reference/command/explain/#mongodb-dbcommand-dbcmd.explain) and determines the amount of information to return.
The possible modes are:
• `"queryPlanner"`
• `"executionStats"`
• `"allPlansExecution"` (Default)
For more information on the modes, see [**explain behavior.**](https://www.mongodb.com/docs/manual/reference/command/explain/#std-label-explain-command-behavior) |
| `comment` | any | Optional.<br>A user-provided comment to attach to this command.<br>Once set, this comment appears alongside records of this command in the following locations:
• [**mongod log messages**](https://www.mongodb.com/docs/manual/reference/log-messages/#std-label-log-messages-ref), in the `attr.command.cursor.comment` field.
• [**Database profiler**](https://www.mongodb.com/docs/manual/reference/database-profiler/#std-label-profiler) output, in the [**`command.comment`**](https://www.mongodb.com/docs/manual/reference/database-profiler/#mongodb-data-system.profile.command) field.
• [**`currentOp`**](https://www.mongodb.com/docs/manual/reference/command/currentOp/#mongodb-dbcommand-dbcmd.currentOp) output, in the [**`command.comment`**](https://www.mongodb.com/docs/manual/reference/command/currentOp/#mongodb-data-currentOp.command) field.
A comment can be any valid [**BSON type**](https://www.mongodb.com/docs/manual/reference/bson-types/#std-label-bson-types) (string, integer, object, array, etc).
If you specify `explain` without a `comment`, it inherits any `comment` in the command specified to `explain`. |

<img src="..\imgs\s10\s10-4.png" width=800 height=400 >
<br>
<img src="..\imgs\s10\s10-5.png" width=800 height=400 >

---

## 10.14 Understanding covered queries

- [Query optimization](https://www.mongodb.com/docs/manual/core/query-optimization/)

In [ ]:
db.customers.insertMany( [ { name: "Max", age: 29, salary: 3000 }, { name: "Manu", age: 30, salary: 4000 } ] )

db.customers.createIndex( { name: 1 } ) // name_1

//  totalKeysExamined: 1,    totalDocsExamined: 1,
db.customers.explain("executionStats").find( { name: "Max" } )

//  totalKeysExamined: 1,    totalDocsExamined: 0,
db.customers.explain("executionStats").find( { name: "Max" }, { _id: 0, name: 1 } )

---

## 10.15 How MongoDB reject plans

- [Query plans](https://www.mongodb.com/docs/manual/core/query-plans/)

> For any given query, the MongoDB query planner chooses and caches the most efficient query plan given the available indexes. 
To evaluate the efficiency of query plans, the query planner runs all candidate plans during a trial period. 
In general, the winning plan is the query plan that produces the most results during the trial period while performing the least amount of work.

<img src="..\imgs\s10\s10-6.png" width=500 height=300 >
<br>
<img src="..\imgs\s10\s10-7.png" width=800 height=200 >

In [ ]:
db.customers.getIndexes()
// resultado:
[
  { v: 2, key: { _id: 1 }, name: '_id_' },
  { v: 2, key: { name: 1 }, name: 'name_1' },
]

// criando novo index:
db.customers.createIndex( { age: 1, name: 1 } ) // age_1_name_1

db.customers.explain().find( { name: "Max", age: 30 } )
// a ordem da query não importa neste caso, pois o index foi criado com AGE primeiro e NAME depois
// este caso gerou um "plano rejeitado", no caso, usar o index "name_1" já existente
 rejectedPlans: [
      {
        isCached: false,
        stage: 'FETCH',
        filter: { age: { '$eq': 30 } },
        inputStage: {
          stage: 'IXSCAN',
          keyPattern: { name: 1 },
          indexName: 'name_1',
          isMultiKey: false,
          multiKeyPaths: { name: [] },
          isUnique: false,
          isSparse: false,
          isPartial: false,
          indexVersion: 2,
          direction: 'forward',
          indexBounds: { name: [ '["Max", "Max"]' ] }
        }
      }
    ]

db.customers.explain("allPlansExecution").find( { name: "Max", age: 30 } ) // details statistics para todos os planos, incluindo os rejeitados

---

## 10.16 Using multi-key indexes

- [Multi-key index](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-multikey/)

> Multikey indexes collect and sort data from fields containing array values. Multikey indexes improve performance for queries on array fields.
> 
> 
> You do not need to explicitly specify the multikey type. When you create an index on a field that contains an array value, MongoDB automatically sets that index to be a multikey index.
> 
> MongoDB can create multikey indexes over arrays that hold both scalar values (for example, strings and numbers) and embedded documents. 
> If an array contains multiple instances of the same value, the index only includes one entry for the value.
>

In [ ]:
db.contacts.drop()

db.contacts.insertOne( { name: "Max", hobbies: [ "Cooking", "Sports" ], addresses: [ { street: "Main Street" }, { street: "Second Street" } ] } )

// index an array:
db.contacts.createIndex( { hobbies: 1 } ) // hobbies_1

db.contacts.createIndex( { addresses: 1 } ) // addresses_1

db.contacts.explain("executionStats").find( { "addresses.street": "Main Street" } ) // used stage: 'COLLSCAN'
// used stage: 'COLLSCAN'
// isso ocorreu pq os nossos indexes contém todos os documentos da coleção

db.contacts.explain("executionStats").find( { addresses: { street: "Main Street" } } ) // stage: 'IXSCAN'

db.contacts.createIndex( { "addresses.street": 1 } ) // also multikey index - > addresses.street_1
db.contacts.explain("executionStats").find( { "addresses.street": "Main Street" } ) // repeat
// agora ele usou o stage: 'IXSCAN'

- Restrictions:

In [ ]:
db.contacts.createIndex( { name: 1, hobbies: 1 } ) // compound index working with multikey index -> name_1_hobbies_1

db.contacts.createIndex( { addresses: 1, hobbies: 1 } ) 
// MongoServerError[CannotIndexParallelArrays]: Index build failed: 080a3818-c7dc-465d-8d82-3173fd0a2bb4: Collection contactData.contacts ( b8450fbf-071a-425a-aefa-1bb02ed8dc55 ) :: 
// caused by :: collection scan stopped. totalRecords: 0; durationMillis: 13ms; phase: collection scan; collectionScanPosition: (None); readSource: kNoTimestamp :: 
// caused by :: cannot index parallel arrays [hobbies] [addresses]

---

## 10.17 Understanding text index

- [Text index](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/)

> Text indexes support [**text search queries**](https://www.mongodb.com/docs/manual/text-search/#std-label-text-search) on fields that contain string content. Text indexes improve performance when you search for specific words or strings within string content.
> 
> 
> A collection can only have **one** text index, but that index may include multiple fields.

> The `$text` operator accepts a text query document with the following fields:
> 
> 
> 
> | **Field** | **Type** | **Description** |
> | --- | --- | --- |
> | `$search` | string | A string of terms that MongoDB parses and uses to query the text index. MongoDB performs a logical `OR` query on the terms unless you specify the terms as an [**exact string**](https://www.mongodb.com/docs/manual/reference/operator/query/text/#std-label-text-operator-exact-string). For more information on the field, see [**Behavior**](https://www.mongodb.com/docs/manual/reference/operator/query/text/#std-label-text-query-operator-behavior). |
> | `$language` | string | Optional. The language that determines the list of stop words for the query and the rules for the stemmer and tokenizer. If unspecified, MongoDB uses the default language of the index. 
> For supported languages, see [**Text Search Languages on Self-Managed Deployments.**](https://www.mongodb.com/docs/manual/reference/text-search-languages/#std-label-text-search-languages)
> If you specify a `default_language` value of `none`, the text index parses through each word in the field, including stop words, and ignores suffix stemming. |
> | `$caseSensitive` | boolean | Optional. A boolean flag to enable or disable case sensitivity. Defaults to `false`. If unspecified, MongoDB defers to the case insensitivity of the [**text**](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/#std-label-index-type-text) index.
> For more information, see [**Case Insensitivity.**](https://www.mongodb.com/docs/manual/reference/operator/query/text/#std-label-text-operator-case-sensitivity) |
> | `$diacriticSensitive` | boolean | Optional. A boolean flag to enable or disable diacritic sensitivity against [**version 3 text indexes**](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/#std-label-index-type-text). Defaults to `false`. If unspecified, MongoDB defers to the diacritic insensitivity of the [**text**](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/#std-label-index-type-text) index.
> Text queries against earlier versions of the text index are inherently diacritic sensitive and cannot be diacritic insensitive. As such, the `$diacriticSensitive` option has no effect with earlier versions of the `text` index.
> For more information, see [**Diacritic Insensitivity.**](https://www.mongodb.com/docs/manual/reference/operator/query/text/#std-label-text-operator-diacritic-sensitivity) |


<br>
<img src="..\imgs\s10\s10-8.png" width=800 height=200 >

- mongoDB removes the “stop words” and storage the key words only

In [ ]:
db.products.insertMany( [ 
	{ title: "A Book", description: "This is an awesome book about an young artist" },
	{ title: "Red T-Shirt", description: "This T-Shirt is red and it's pretty awesome" }
] )

db.products.createIndex( { description: "text" } ) // description_text

db.products.find( { $text: { $search: "awesome" } } ) // is not case-sensitive as everything is storage as lower case

db.products.find( { $text: { $search: "book" } } )

db.products.find( { $text: { $search: "red book" } } ) // retorna ambos os documentos, mesmo que cada palavra esteja individualmente em cada um deles

db.products.find( { $text: { $search: "\"awesome book\"" } } ) // o operador "\" é usado para procurar pela frase exata

---

## 10.18 Text indexes and sorting

- https://www.mongodb.com/docs/manual/reference/operator/aggregation/meta/ 

> Returns the metadata associated with a document, e.g. `"textScore"` when performing text search.
> 
> 
> 
> | **Keyword** | **Description** |
> | --- | --- |
> | `"textScore"` | Returns the score associated with the corresponding [**`$text`**](https://www.mongodb.com/docs/manual/reference/operator/query/text/#mongodb-query-op.-text) query for each matching document.<br>The text score signifies how well the document matched the [**search term or terms.**](https://www.mongodb.com/docs/manual/reference/operator/query/text/#std-label-match-operation-stemmed-words)
> `{ $meta: "textScore" }` must be used in conjunction with a `$text` query.
> In earlier versions, if not used in conjunction with a `$text` query, returns a score of null.
> `$text` provides text query capabilities for self-managed (non-Atlas) deployments. For data hosted on MongoDB Atlas, MongoDB offers an improved full-text query solution, [**Atlas Search.**](https://www.mongodb.com/docs/atlas/atlas-search/) |
> | `"indexKey"` | Returns an index key for the document if a non-[**text**](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/#std-label-index-type-text) index is used. <br>The `{ $meta: "indexKey" }` expression is for debugging purposes only, and not for application logic, and is preferred over [**`cursor.returnKey()`.**](https://www.mongodb.com/docs/manual/reference/method/cursor.returnKey/#mongodb-method-cursor.returnKey) |

In [ ]:
db.products.find( { $text: { $search: "awesome T-shirt" } } )

db.products.find( { $text: { $search: "awesome T-shirt" } }, { score: { $meta: "textScore"} } )

db.products.find( { $text: { $search: "awesome T-shirt" } }, { score: { $meta: "textScore"} } ).sort( { score: { $meta: "textScore" } } )

---

## 10.19 Creating combined text index

- [Create a compound text index](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/create-text-index/#create-a-compound-text-index)

> You can index multiple fields in a single text index. A text index can contain up to 32 fields. To see an example, see [**Create a Compound Text Index.**](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/create-text-index/#std-label-compound-text-index-example)
>

In [ ]:
db.products.getIndexes()

db.products.dropIndex( "description_text" ) // drops by index name

db.products.createIndex( { title: "text", description: "text" } ) // ainda existirá somente 1 "text-index" mas ele conterá o conteúdo dos 2 campos -> title_text_description_text

db.products.insertOne( { title: "A Ship", description: "Floats perfectly!" } )

db.products.find( { $text: { $search: "ship" } } )

---

## 10.20 Using text indexes to exclude words

In [ ]:
// exemplo: retornar todos os resultados para "awesome" onde não exista tbm a palavra "t-shirt"
db.products.find( { $text: { $search: "awesome -t-shirt" } } ) // -<word> significa que aquela palavra deve ser excluída da busca

---

## 10.21 Setting the default language and using weights

- [Specify the Default Language for a Text Index on Self-Managed Deployments](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/specify-text-index-language/#specify-the-default-language-for-a-text-index-on-self-managed-deployments)

> By default, the `default_language` for text indexes is `english`. To improve the performance of non-English text search queries, you can specify a different default language associated with your text index.
> 
> 
> The default language associated with the indexed data determines the suffix stemming rules. The default language also determines which language-specific stop words (for example, `the`, `an`, `a`, and `and` in English) are not indexed.
> 
> To specify a different language, use the `default_language` option when creating the text index. To see the languages available for text indexing, see [**Text Search Languages on Self-Managed Deployments**](https://www.mongodb.com/docs/manual/reference/text-search-languages/#std-label-text-search-languages).
>

In [ ]:
db.products.getIndexes()
db.products.dropIndex( "title_text_description_text" )

db.products.createIndex(
	{ title: "text", 
		description: "text"
	}, 
	{ default_language: "english", 
		weights: 
			{ title: 1, 
				description: 10 } 
	}
) // title_text_description_text

db.products.find( { $text: { $search: "red" } } )

db.products.find( { $text: { $search: "red" } }, { score: { $meta: "textScore" } } ) // score: 6.666666666666667

- [Assign Weights to Text Search Results on Self-Managed Deployments](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-text/control-text-search-results/#assign-weights-to-text-search-results-on-self-managed-deployments)

> When MongoDB returns text search results, it assigns a **score** to each returned document. The score indicates the relevance of the document to a given search query. 
You can sort returned documents by score to have the most relevant documents appear first in the result set.
> 
> 
> If you have a [**compound index**](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-compound/#std-label-index-type-compound) with multiple text index keys, you can specify different **weights** for each indexed field. 
> The weight of an indexed field indicates the significance of the field relative to the other indexed fields, with higher weights resulting in higher text search scores.
> 
> For example, you can emphasize search matches on a `title` field if you know users are likely to search for titles, or if `title` contains more relevant search terms compared to other document fields.
>

---

## 10.22 Building indexes

- [Index Builds on Populated Collections](https://www.mongodb.com/docs/manual/core/index-creation/#index-builds-on-populated-collections)

<img src="..\imgs\s10\s10-9.png" width=600 height=200 >


> **Comparison to Foreground and Background Builds**
> 
> 
> Previous versions of MongoDB supported building indexes either in the foreground or background. Foreground index builds were fast and produced more efficient index data structures, but required blocking all read-write access to the parent database of the collection being indexed for the duration of the build. Background index builds were slower and had less efficient results, but allowed read-write access to the database and its collections during the build process.
> 
> Index builds now obtain an exclusive lock on only the collection being indexed during the start and end of the build process to protect metadata changes. The rest of the build process uses the yielding behavior of background index builds to maximize read-write access to the collection during the build. Index builds still produce efficient index data structures despite the more permissive locking behavior.
> 
> The optimized index build performance is at least on par with background index builds. For workloads with few or no updates received during the build process, optimized index builds can be as fast as a foreground index build on that same data.
> 
> Use [**`db.currentOp()`**](https://www.mongodb.com/docs/manual/reference/method/db.currentOp/#mongodb-method-db.currentOp) to monitor the progress of ongoing index builds.
> 
> MongoDB ignores the `background` index build option if specified to [**`createIndexes`**](https://www.mongodb.com/docs/manual/reference/command/createIndexes/#mongodb-dbcommand-dbcmd.createIndexes) or its shell helpers [**`createIndex()`**](https://www.mongodb.com/docs/manual/reference/method/db.collection.createIndex/#mongodb-method-db.collection.createIndex) and [**`createIndexes()`.**](https://www.mongodb.com/docs/manual/reference/method/db.collection.createIndexes/#mongodb-method-db.collection.createIndexes)
>

In [ ]:
## no cmd, dentro da pasta onde está o arquivo credit-rating.js:
mongosh credit-rating.js

In [ ]:
// use credit
db.ratings.findOne()

db.ratings.createIndex( { age: 1 } ) // age_1 -> demora um pouco para ser criado pois temos 1bi de documentos

db.ratings.explain("executionStats").find( { age: { $gt: 80 } } ) // stage: 'IXSCAN', executionTimeMillis: 200

db.ratings.dropIndex( { age: 1 } )

db.ratings.explain("executionStats").find( { age: { $gt: 80 } } ) //  stage: 'COLLSCAN', executionTimeMillis: 870

db.ratings.dropIndex( { age: 1 } )
db.ratings.createIndex( { age: 1 }, { background: true } ) // age_1

---

## 10.24 Wrap up

<img src="..\imgs\s10\s10-10.png" width=700 height=400 >

---

## 10.25 Useful resources and links

> Helpful Articles/ Docs:
> 
> - More on partialFilterExpressions: https://docs.mongodb.com/manual/core/index-partial/
> - Supported default_languages: https://docs.mongodb.com/manual/reference/text-search-languages/#text-search-languages
> - How to use different languages in the same index: https://docs.mongodb.com/manual/tutorial/specify-language-for-text-index/#create-a-text-index-for-a-collection-in-multiple-languages